### Project 1

TimeZone Class

In [2]:
import numbers
import itertools
from datetime import timedelta

In [4]:
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0: 
            raise ValueError('TimeZone name cannot be empty')

        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minutes offset must be an integer")

        if abs(offset_minutes) > 59 :
            raise ValueError("Minutes offset must be [-59, 59].")

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError("Offset must be between -12.00 and 14.00")

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return(isinstance(other, TimeZone) and
               self.name == other.name and
               self._offest_hours == other._offset_hours and
               self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return(f"TimeZone(name='{self.name}', "
               f"offest_hours={self._offset_hours}, "
               f"offset_minutes={self._offset_minutes})")


In [12]:
class Account:
    transaction_counter = itertools.count(100)

    interset_rate = 0.5 # percent

    def __init__(self, account_number, first_name, last_name, timezone=None, 
                initial_balance=0):
        self._account_number = account_number
        self._first_name = first_name
        self._last_name = last_name
        # print("Account is created!")

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone

        self._balance = float(initial_balance)

    @property
    def account_number(self):
        return self._account_number

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("First name cannot be empty!")
        # self._first_name = value
        self._first_name = self.validate_and_set_name('_first_name', value, "First Name")

    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("Last name cannot be empty!")
        # self._last_name = value
        self._last_name = self.validate_and_set_name("_last_name", value, "Last Name")

    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone

    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time must be valid')
        self._timezone = value
        
        
    # @staticmethod
    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{field_title} cannot be empty!")
        # return str(value).strip()
        setattr(self, attr_name, value)

In [15]:
class Account:
    transaction_counter = itertools.count(100)

    _interset_rate = 0.5 # percent

    def __init__(self, account_number, first_name, last_name, timezone=None, 
                initial_balance=0):
        self._account_number = account_number
        self._first_name = first_name
        self._last_name = last_name
        # print("Account is created!")

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone

        self._balance = float(initial_balance)

    @property
    def account_number(self):
        return self._account_number

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("First name cannot be empty!")
        # self._first_name = value
        self._first_name = self.validate_and_set_name('_first_name', value, "First Name")

    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("Last name cannot be empty!")
        # self._last_name = value
        self._last_name = self.validate_and_set_name("_last_name", value, "Last Name")

    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone

    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time must be valid')
        self._timezone = value

    @classmethod
    def get_iterest_rate(cls):
        return cls._interset_rate

    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError("Interest rate must be a real number")

        if value < 0:
            raise ValueError("must be above zero")

        cls._interset_rate = value

    # @staticmethod
    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{field_title} cannot be empty!")
        # return str(value).strip()
        setattr(self, attr_name, value)

In [16]:
Account.get_iterest_rate()

0.5

In [19]:
Account.set_interest_rate(-11)
Account.get_iterest_rate()

ValueError: must be above zero

In [10]:
try:
    a = Account(123, "July", "Clark", "-7.00")
except ValueError as ex:
    print(ex)

Time must be valid


In [13]:
b = Account('23211', "Bob", "Dilan")


In [14]:
b.interset_rate

0.5

In [8]:
b.full_name

'Bob Dilan'

In [3]:
tz1 = TimeZone("ABC", -2, -15)
tz1.name

'ABC'

In [4]:
from datetime import datetime

dt = datetime.utcnow()
print(dt)

2023-05-10 01:09:41.742426


In [5]:
print(dt + tz1.offset)

2023-05-09 22:54:41.742426


In [6]:
try:
    tz = TimeZone("", 1, 10)
except ValueError as ex:
    print(ex)

TimeZone name cannot be empty


In [8]:
class TransactionID:
    def __init__(self, start_id):
        self._start_id = start_id

    def next(self):
        self._start_id += 1
        return self._start_id

In [9]:
class Account:
    transaction_counter = TransactionID(100)

    def make_transaction(self):
        new_trans_id = Account.transaction_counter.next()
        return new_trans_id

In [14]:
ac1 = Account()
ac2 = Account()

In [15]:
print(ac1.make_transaction())
print(ac2.make_transaction())

101
102


In [12]:
def transaction_ids(start_id):
    while True:
        start_id += 1
        yield start_id

In [13]:
class Account:
    transaction_counter = transaction_ids(100)

    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id

Using module `itertools`

In [17]:
import itertools

class Account:
    transaction_counter = itertools.count(100)

    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id

In [18]:
ac1 = Account()
ac2 = Account()

print(ac1.make_transaction())
print(ac2.make_transaction())
print(ac2.make_transaction())

100
101
102
